In [1]:
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import pandas as pd
import numpy as np

In [2]:
path = os.getcwd() + os.sep + 'data' + os.sep

activities = pd.read_json(path + 'activities.json')

movie_activities = activities[activities['category'] == 'movie']

show_activities =  activities[activities['category'] == 'show']

movie_activities.drop(columns = 'category', inplace = True)
show_activities.drop(columns = 'category', inplace = True)

movie_activities

<ipython-input-2-b9fd80230a0a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_activities.drop(columns = 'category', inplace = True)
<ipython-input-2-b9fd80230a0a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_activities.drop(columns = 'category', inplace = True)


,content_id,activity,user_id,datetime
0,17,Play,20,2023-04-02 19:34:17.597077
6,315,Dislike,10,2023-04-02 19:34:17.597126
7,44,Play,14,2023-04-02 19:34:17.597131
8,55,Like,13,2023-04-02 19:34:17.597136
9,246,Dislike,16,2023-04-02 19:34:17.597142
...,...,...,...,...
64995,144,Play,11,2023-04-02 19:34:22.360340
64996,122,Play,0,2023-04-02 19:34:22.360344
64997,256,Play,2,2023-04-02 19:34:22.360348
64998,273,Like,2,2023-04-02 19:34:22.360351


In [3]:
n_users = activities['user_id'].nunique()
n_users

31

## Shows

In [4]:
show_activities

,content_id,activity,user_id,datetime
1,10402,Play,11,2023-04-02 19:34:17.597095
2,10563,Like,20,2023-04-02 19:34:17.597102
3,10159,Like,15,2023-04-02 19:34:17.597109
4,10092,Like,17,2023-04-02 19:34:17.597114
5,10445,Dislike,10,2023-04-02 19:34:17.597120
...,...,...,...,...
69995,10108,Like,26,2023-04-02 19:34:22.906881
69996,10302,Play,19,2023-04-02 19:34:22.906895
69997,10166,Dislike,21,2023-04-02 19:34:22.906910
69998,10048,Like,0,2023-04-02 19:34:22.906925


In [5]:
# Variables for adjusting formula

clicks = 0.3
like = 10
dislike = -10

# current formula is clicks * 0.3 +/-10 (+10 for like and -10 for dislike)

In [6]:
likes_dislikes = show_activities[(show_activities['activity'] == 'Like') | (show_activities['activity'] == 'Dislike')]

likes_dislikes.sort_values('datetime', inplace = True)

latest = likes_dislikes.groupby(['user_id', 'content_id']).last()

latest.drop(columns = 'datetime', inplace = True)

unstacked = latest.unstack()

unstacked.columns = unstacked.columns.droplevel()

unstacked.replace({'Dislike':dislike, 'Like':like}, inplace = True)

unstacked.fillna(0, inplace = True)

unstacked

<ipython-input-6-f35828a35cb1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  likes_dislikes.sort_values('datetime', inplace = True)


content_id,10000,10001,10002,10003,10004,10005,10006,10007,10008,10009,...,10614,10615,10616,10617,10618,10619,10620,10621,10622,10623
user_id,,,,,,,,,,,,,,,,,,,,,
0,10.0,10.0,-10.0,10.0,-10.0,10.0,10.0,-10.0,10.0,-10.0,...,10.0,10.0,-10.0,-10.0,0.0,10.0,10.0,10.0,-10.0,0.0
1,10.0,10.0,0.0,-10.0,0.0,10.0,10.0,10.0,0.0,10.0,...,10.0,0.0,-10.0,10.0,10.0,10.0,10.0,10.0,-10.0,0.0
2,10.0,10.0,-10.0,-10.0,0.0,10.0,10.0,-10.0,0.0,10.0,...,10.0,0.0,0.0,0.0,0.0,-10.0,10.0,10.0,-10.0,0.0
3,10.0,0.0,10.0,10.0,-10.0,0.0,10.0,-10.0,10.0,10.0,...,10.0,10.0,10.0,-10.0,-10.0,10.0,0.0,10.0,-10.0,0.0
4,10.0,-10.0,10.0,-10.0,10.0,0.0,0.0,10.0,10.0,-10.0,...,0.0,0.0,10.0,10.0,10.0,10.0,0.0,10.0,10.0,0.0
5,0.0,10.0,-10.0,0.0,10.0,0.0,10.0,0.0,0.0,10.0,...,0.0,-10.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
6,-10.0,0.0,-10.0,-10.0,10.0,-10.0,10.0,10.0,0.0,0.0,...,10.0,-10.0,0.0,10.0,0.0,10.0,10.0,10.0,10.0,0.0
7,-10.0,10.0,-10.0,10.0,0.0,10.0,0.0,10.0,-10.0,0.0,...,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,-10.0,0.0
8,0.0,10.0,10.0,10.0,10.0,0.0,0.0,10.0,10.0,10.0,...,0.0,-10.0,-10.0,10.0,-10.0,10.0,-10.0,10.0,0.0,0.0


In [7]:
max_length = list(unstacked.columns)[-1] + 1

max_length

10624

In [8]:
# turning it into sparse matrix

likes_matrix = np.zeros((n_users, max_length))

users = [i for i in list(unstacked.index)]
cols = list(unstacked.columns)

for i in users:
    for j in cols:
        likes_matrix[i, j] = unstacked.loc[i, j]
        
likes_matrix

array([[  0.,   0.,   0., ...,  10., -10.,   0.],
       [  0.,   0.,   0., ...,  10., -10.,   0.],
       [  0.,   0.,   0., ...,  10., -10.,   0.],
       ...,
       [  0.,   0.,   0., ...,  10.,   0.,   0.],
       [  0.,   0.,   0., ..., -10.,  10.,   0.],
       [  0.,   0.,   0., ...,  10.,  10.,   0.]])

In [9]:
# creating sparse matrix for plays

plays = show_activities[show_activities['activity'] == 'Play']

grouped_plays = plays.groupby(['user_id', 'content_id']).count()

pivot_plays = grouped_plays.pivot_table(index='user_id', columns = 'content_id', values = 'activity')

pivot_plays.fillna(0, inplace = True)

plays_matrix = np.zeros((n_users, max_length))

users = [i for i in range(n_users)]
cols = list(pivot_plays.columns)

for i in users:
    for j in cols:
        plays_matrix[i, j-1] = pivot_plays.loc[i, j] * clicks
        
plays_matrix

array([[0. , 0. , 0. , ..., 0.3, 0. , 0. ],
       [0. , 0. , 0. , ..., 0.3, 0. , 0. ],
       [0. , 0. , 0. , ..., 0.3, 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0.3, 0. , 0. ],
       [0. , 0. , 0. , ..., 0.3, 0. , 0. ]])

In [10]:
scores_matrix = plays_matrix + likes_matrix

scores_df = pd.DataFrame(scores_matrix)

scores_df

,0,1,2,3,4,5,6,7,8,9,...,10614,10615,10616,10617,10618,10619,10620,10621,10622,10623
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.6,10.3,-10.0,-9.4,0.0,10.3,10.3,10.3,-10.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.3,-10.0,10.0,10.6,10.3,10.0,10.3,-10.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.3,-10.0,10.0,10.3,-10.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,10.0,10.3,-9.7,-9.4,10.6,0.0,10.6,-10.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.3,10.0,10.0,10.0,0.0,10.3,10.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.6,-9.7,0.0,10.6,10.0,10.6,10.3,10.0,10.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,-9.1,0.6,10.3,0.3,10.3,10.3,10.0,10.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.3,10.6,10.3,10.0,10.0,10.6,10.0,-10.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.2,-9.7,-9.4,10.6,-9.4,10.3,-9.7,10.6,0.3,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-9.1,-10.0,-9.7,10.0,10.0,0.6,10.3,10.3,0.0


In [11]:
scores_df_clean = scores_df.copy().loc[:, ~(scores_df == 0).all()]

scores_df_clean

,9999,10000,10001,10002,10003,10004,10005,10006,10007,10008,...,10614,10615,10616,10617,10618,10619,10620,10621,10622,10623
0,0.9,10.6,10.6,-9.1,10.0,-10.0,10.3,10.3,-9.7,10.3,...,10.6,10.3,-10.0,-9.4,0.0,10.3,10.3,10.3,-10.0,0.0
1,0.3,10.3,10.3,0.0,-9.7,0.3,10.0,10.6,10.3,0.0,...,10.0,0.3,-10.0,10.0,10.6,10.3,10.0,10.3,-10.0,0.0
2,0.0,10.3,10.3,-10.0,-9.4,0.6,10.3,10.3,-10.0,0.0,...,10.0,0.0,0.0,0.0,0.3,-10.0,10.0,10.3,-10.0,0.0
3,0.6,10.0,0.3,10.6,10.6,-9.1,0.0,10.6,-9.7,10.0,...,10.0,10.0,10.3,-9.7,-9.4,10.6,0.0,10.6,-10.0,0.0
4,0.0,10.3,-9.7,10.0,-10.0,10.6,0.6,0.3,10.3,10.3,...,0.0,0.0,10.3,10.0,10.0,10.0,0.0,10.3,10.0,0.0
5,0.0,0.3,10.3,-9.7,0.6,10.3,0.0,10.0,0.6,0.3,...,0.6,-9.7,0.0,10.6,10.0,10.6,10.3,10.0,10.0,0.0
6,0.0,-10.0,0.6,-10.0,-10.0,10.3,-9.1,10.0,10.0,0.0,...,10.0,-9.1,0.6,10.3,0.3,10.3,10.3,10.0,10.0,0.0
7,0.6,-10.0,10.3,-9.7,10.6,0.3,10.3,0.9,10.0,-9.7,...,0.0,10.3,10.6,10.3,10.0,10.0,10.6,10.0,-10.0,0.0
8,0.6,0.6,10.0,10.3,10.6,10.0,0.0,0.0,11.2,10.0,...,1.2,-9.7,-9.4,10.6,-9.4,10.3,-9.7,10.6,0.3,0.0
9,0.3,10.3,10.0,-10.0,10.0,10.0,0.3,-9.7,10.3,-10.0,...,-10.0,-9.1,-10.0,-9.7,10.0,10.0,0.6,10.3,10.3,0.0


In [12]:
scores_df_clean.replace(0, np.nan, inplace = True)

In [13]:
# Compute cosine similarity between all pairs of users
user_similarities = cosine_similarity(scores_matrix)


In [14]:
# Function to predict a user's rating for an item
def predict_rating(user_item_matrix, user_id, item_id):
    # Find the most similar users to the given user
    similar_users = np.argsort(user_similarities[user_id])[::-1][1:]
    
    # Compute weighted average of their ratings for the item 
    weighted_sum = 0
    weight_sum = 0   
    for sim_user in similar_users:
        if user_item_matrix[sim_user][item_id] != 0:
            similarity = user_similarities[user_id][sim_user]
            rating = user_item_matrix[sim_user][item_id]
            weighted_sum += similarity * rating            
            weight_sum += similarity   
    if weight_sum == 0:
        return 0
    else:
        return weighted_sum / weight_sum
    # Example usage:
    # Predict user 0's rating for item 2
    #predicted_rating = predict_rating(0, 2)
    #print(predicted_rating)

In [15]:
missings = np.where(scores_matrix == 0)

u_ids = missings[0]
c_ids = missings[1]

c_ids

array([    0,     1,     2, ..., 10593, 10594, 10623], dtype=int64)

In [16]:
z = np.zeros((n_users, max_length))

for i in range(len(u_ids)):
    z[u_ids[i], c_ids[i]] = predict_rating(scores_matrix, u_ids[i], c_ids[i])
        
z

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 10.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 10.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 10.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         2.14680817, 10.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 10.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 10.        ]])

In [17]:
only_predictions = pd.DataFrame(z)

only_predictions

,0,1,2,3,4,5,6,7,8,9,...,10614,10615,10616,10617,10618,10619,10620,10621,10622,10623
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,3.734774,0.000000,0.000000,0.000000,0.000000,10.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,4.796473,0.841591,4.740752,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.628294,0.000000,0.000000,10.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.760076,3.065448,0.000000,0.000000,0.000000,0.000000,5.017022,0.000000,0.000000,10.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,1.731278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.604979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0


In [18]:
unwatched_list = []

for i in range(len(only_predictions)):
    unwatched_list.append(list(np.where(z[i] != 0)[0]))
                     
unwatched_list

[[10025,
  10027,
  10028,
  10036,
  10065,
  10068,
  10076,
  10079,
  10103,
  10110,
  10123,
  10137,
  10161,
  10163,
  10164,
  10187,
  10203,
  10208,
  10212,
  10220,
  10247,
  10253,
  10264,
  10274,
  10280,
  10295,
  10309,
  10315,
  10324,
  10327,
  10331,
  10343,
  10354,
  10363,
  10408,
  10416,
  10417,
  10433,
  10456,
  10485,
  10494,
  10498,
  10503,
  10506,
  10518,
  10525,
  10536,
  10552,
  10554,
  10556,
  10571,
  10584,
  10588,
  10594,
  10601,
  10618,
  10623],
 [10002,
  10008,
  10026,
  10045,
  10058,
  10068,
  10099,
  10101,
  10107,
  10133,
  10137,
  10140,
  10143,
  10159,
  10164,
  10165,
  10173,
  10185,
  10198,
  10202,
  10209,
  10213,
  10236,
  10240,
  10247,
  10254,
  10258,
  10280,
  10293,
  10309,
  10330,
  10335,
  10339,
  10358,
  10361,
  10363,
  10367,
  10368,
  10374,
  10379,
  10385,
  10396,
  10407,
  10417,
  10424,
  10426,
  10445,
  10456,
  10467,
  10469,
  10487,
  10496,
  10498,
  10512,


In [19]:
score_list = []
c = 0
for i in unwatched_list:
    s = []
    for j in i:
        s.append(z[c, j])
    score_list.append(s)
    c += 1
    
score_list

[[3.6176190242835435,
  3.3033090736882533,
  2.68220827235607,
  2.407106335376867,
  2.1380717765148396,
  6.771119542292902,
  4.394300000786201,
  2.8027468532452624,
  3.8294079425096723,
  1.7613242749894933,
  2.590792339713097,
  0.9897960657428753,
  2.318450860587923,
  5.5899073521589555,
  0.15090497049118703,
  5.159511656079505,
  4.073331111515387,
  2.8989052686079075,
  4.208855390860275,
  3.0654161301901306,
  3.8804749405172467,
  -0.04201764111336163,
  3.711623767415477,
  -0.4667095187918788,
  3.1259140450279537,
  2.0696099330125555,
  1.4968054919130889,
  5.192070637246127,
  3.606865799899913,
  -0.1648193452775408,
  2.2170748314195605,
  5.4249712394925895,
  0.33579045897709164,
  4.463414012945693,
  1.9193922295896457,
  3.9545243747359424,
  6.384725898878912,
  4.576766729871928,
  3.0198900558510497,
  1.5509483824077348,
  3.1974698524099465,
  5.124441315678129,
  3.1723609848505125,
  3.797709060912715,
  2.2537102761224297,
  5.3113117519550475,


In [20]:
score_indices = []
for i in score_list:
    order = sorted(range(len(i)), reverse = True, key=lambda k: i[k])
    score_indices.append(order)
    
score_indices

[[56,
  53,
  5,
  36,
  13,
  31,
  45,
  27,
  15,
  41,
  37,
  33,
  48,
  6,
  18,
  16,
  35,
  20,
  8,
  43,
  55,
  22,
  0,
  28,
  1,
  40,
  42,
  24,
  19,
  38,
  50,
  17,
  7,
  2,
  10,
  49,
  3,
  51,
  12,
  44,
  30,
  4,
  25,
  34,
  9,
  52,
  39,
  26,
  11,
  46,
  54,
  32,
  14,
  47,
  21,
  29,
  23],
 [62,
  5,
  22,
  6,
  12,
  43,
  9,
  55,
  52,
  54,
  13,
  39,
  28,
  31,
  7,
  24,
  40,
  44,
  30,
  20,
  35,
  1,
  27,
  56,
  61,
  11,
  8,
  15,
  47,
  42,
  2,
  23,
  49,
  59,
  51,
  57,
  36,
  16,
  45,
  46,
  34,
  4,
  53,
  26,
  41,
  21,
  25,
  17,
  33,
  50,
  19,
  29,
  32,
  38,
  10,
  14,
  60,
  48,
  37,
  18,
  58,
  3,
  0],
 [65,
  23,
  52,
  7,
  42,
  22,
  37,
  51,
  26,
  25,
  10,
  8,
  14,
  35,
  56,
  62,
  64,
  59,
  15,
  36,
  49,
  20,
  40,
  34,
  3,
  21,
  44,
  24,
  53,
  60,
  50,
  54,
  57,
  55,
  47,
  39,
  16,
  4,
  11,
  1,
  48,
  41,
  5,
  27,
  31,
  61,
  46,
  28,
  13,
  12,
  45

In [21]:
rec_list = []
for i in range(len(score_indices)):
    l = unwatched_list[i]
    order = score_indices[i]
    rec_list.append([l[j] for j in order])
    
rec_list

[[10623,
  10594,
  10068,
  10417,
  10163,
  10343,
  10525,
  10315,
  10187,
  10498,
  10433,
  10363,
  10554,
  10076,
  10212,
  10203,
  10416,
  10247,
  10103,
  10506,
  10618,
  10264,
  10025,
  10324,
  10027,
  10494,
  10503,
  10280,
  10220,
  10456,
  10571,
  10208,
  10079,
  10028,
  10123,
  10556,
  10036,
  10584,
  10161,
  10518,
  10331,
  10065,
  10295,
  10408,
  10110,
  10588,
  10485,
  10309,
  10137,
  10536,
  10601,
  10354,
  10164,
  10552,
  10253,
  10327,
  10274],
 [10623,
  10068,
  10236,
  10099,
  10143,
  10417,
  10133,
  10557,
  10498,
  10555,
  10159,
  10379,
  10293,
  10335,
  10101,
  10247,
  10385,
  10424,
  10330,
  10209,
  10363,
  10008,
  10280,
  10570,
  10610,
  10140,
  10107,
  10165,
  10456,
  10407,
  10026,
  10240,
  10469,
  10586,
  10496,
  10574,
  10367,
  10173,
  10426,
  10445,
  10361,
  10058,
  10512,
  10258,
  10396,
  10213,
  10254,
  10185,
  10358,
  10487,
  10202,
  10309,
  10339,
  10374,


In [22]:
# output dataframe with user ids a

recommendations_shows = pd.DataFrame(rec_list)

recommendations_shows.to_csv('show_recommendations.csv', index = False)
recommendations_shows

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,10623,10594,10068,10417,10163,10343,10525,10315,10187,10498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10623,10068,10236,10099,10143,10417,10133,10557,10498,10555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10623,10229,10528,10070,10440,10227,10364,10525,10267,10247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10623,10455,10403,10143,10553,10049,10620,10068,10231,10156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10623,10229,10520,10376,10343,10483,10596,10296,10620,10614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10623,10229,10097,10403,10596,10074,10245,10075,10197,10073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10623,10395,10379,10553,10315,10043,10285,10528,10392,10570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10623,10049,10403,10553,10560,10021,10019,10281,10501,10070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10623,10417,10059,10135,10285,10532,10239,10067,10103,10340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10623,10066,10049,10596,10010,10108,10255,10281,10093,10483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
pd.read_csv('show_recommendations.csv')

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,10623,10594,10068,10417,10163,10343,10525,10315,10187,10498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10623,10068,10236,10099,10143,10417,10133,10557,10498,10555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10623,10229,10528,10070,10440,10227,10364,10525,10267,10247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10623,10455,10403,10143,10553,10049,10620,10068,10231,10156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10623,10229,10520,10376,10343,10483,10596,10296,10620,10614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10623,10229,10097,10403,10596,10074,10245,10075,10197,10073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10623,10395,10379,10553,10315,10043,10285,10528,10392,10570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10623,10049,10403,10553,10560,10021,10019,10281,10501,10070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10623,10417,10059,10135,10285,10532,10239,10067,10103,10340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10623,10066,10049,10596,10010,10108,10255,10281,10093,10483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Movies 

In [24]:
# getting likes/dislikes
likes_dislikes = movie_activities[(movie_activities['activity'] == 'Like') | (movie_activities['activity'] == 'Dislike')]

# latest activity at the bottom
likes_dislikes.sort_values('datetime', inplace = True)

# filtering out all like/dislike that is not at the bottom
latest = likes_dislikes.groupby(['user_id', 'content_id']).last()


# getting the right shape
latest.drop(columns = 'datetime', inplace = True)

unstacked = latest.unstack()

unstacked.columns = unstacked.columns.droplevel()

unstacked.replace({'Dislike':dislike, 'Like':like}, inplace = True)

unstacked.fillna(0, inplace = True)

unstacked

<ipython-input-24-e70775679e06>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  likes_dislikes.sort_values('datetime', inplace = True)


content_id,0,1,2,3,4,5,6,7,8,9,...,343,344,345,346,347,348,349,350,351,352
user_id,,,,,,,,,,,,,,,,,,,,,
0,10.0,10.0,-10.0,10.0,10.0,0.0,10.0,-10.0,0.0,0.0,...,10.0,10.0,10.0,0.0,-10.0,10.0,-10.0,10.0,10.0,0.0
1,10.0,10.0,0.0,-10.0,10.0,10.0,10.0,10.0,10.0,0.0,...,10.0,-10.0,0.0,10.0,10.0,10.0,10.0,-10.0,-10.0,0.0
2,0.0,0.0,10.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,...,0.0,10.0,-10.0,10.0,-10.0,-10.0,10.0,10.0,10.0,0.0
3,-10.0,-10.0,10.0,10.0,10.0,-10.0,-10.0,10.0,10.0,0.0,...,0.0,10.0,10.0,0.0,0.0,0.0,-10.0,10.0,0.0,-10.0
4,10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,10.0,10.0,10.0,...,0.0,-10.0,10.0,0.0,0.0,0.0,10.0,-10.0,10.0,0.0
5,10.0,10.0,0.0,-10.0,10.0,0.0,10.0,10.0,10.0,-10.0,...,10.0,10.0,-10.0,10.0,10.0,-10.0,-10.0,10.0,10.0,10.0
6,10.0,10.0,-10.0,0.0,-10.0,0.0,10.0,0.0,-10.0,0.0,...,10.0,-10.0,10.0,10.0,-10.0,10.0,10.0,0.0,-10.0,0.0
7,10.0,-10.0,0.0,10.0,10.0,10.0,0.0,0.0,-10.0,10.0,...,10.0,0.0,10.0,-10.0,10.0,10.0,10.0,0.0,10.0,0.0
8,10.0,0.0,10.0,10.0,-10.0,0.0,0.0,10.0,0.0,10.0,...,0.0,10.0,10.0,-10.0,10.0,-10.0,0.0,10.0,10.0,0.0


In [25]:
max_length = list(unstacked.columns)[-1] + 1

max_length

353

In [26]:
# turning it into sparse matrix

likes_matrix = np.zeros((n_users, max_length))

users = [i for i in list(unstacked.index)]
cols = list(unstacked.columns)

for i in users:
    for j in cols:
        likes_matrix[i, j] = unstacked.loc[i, j]
        
likes_matrix

array([[ 10.,  10., -10., ...,  10.,  10.,   0.],
       [ 10.,  10.,   0., ..., -10., -10.,   0.],
       [  0.,   0.,  10., ...,  10.,  10.,   0.],
       ...,
       [  0.,  10.,  10., ..., -10.,   0.,   0.],
       [-10.,  10.,  10., ...,   0., -10.,   0.],
       [  0.,  10.,  10., ...,  10.,   0.,   0.]])

In [27]:
# creating sparse matrix for plays

plays = movie_activities[movie_activities['activity'] == 'Play']

grouped_plays = plays.groupby(['user_id', 'content_id']).count()

pivot_plays = grouped_plays.pivot_table(index='user_id', columns = 'content_id', values = 'activity')

pivot_plays.fillna(0, inplace = True)

plays_matrix = np.zeros((n_users, max_length))

users = [i for i in range(n_users)]
cols = list(pivot_plays.columns)

for i in users:
    for j in cols:
        plays_matrix[i, j] = pivot_plays.loc[i, j] * clicks
        
plays_matrix

array([[0. , 0.6, 0.3, ..., 0.3, 0.3, 0. ],
       [0.6, 0.3, 0.3, ..., 0.3, 0.6, 0. ],
       [0. , 0.3, 0.3, ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0.3, ..., 0.3, 0.6, 0. ],
       [0. , 0. , 0.3, ..., 0. , 0.6, 0. ],
       [0.3, 0.3, 0.9, ..., 0.6, 0. , 0. ]])

In [28]:
scores_matrix = plays_matrix + likes_matrix

scores_df = pd.DataFrame(scores_matrix)

scores_df

,0,1,2,3,4,5,6,7,8,9,...,343,344,345,346,347,348,349,350,351,352
0,10.0,10.6,-9.7,10.6,10.0,0.0,10.3,-9.7,0.0,0.6,...,10.0,10.3,10.6,0.0,-10.0,10.3,-10.0,10.3,10.3,0.0
1,10.6,10.3,0.3,-9.7,10.0,10.6,10.6,10.0,10.9,0.0,...,10.0,-9.4,0.0,10.0,10.3,10.0,10.6,-9.7,-9.4,0.0
2,0.0,0.3,10.3,0.0,0.6,0.3,0.6,10.0,10.3,0.3,...,0.0,10.3,-9.7,10.3,-9.7,-9.7,10.0,10.0,10.0,0.0
3,-10.0,-10.0,10.0,10.6,10.0,-10.0,-9.7,10.0,10.6,0.0,...,0.3,11.2,10.0,0.3,0.3,0.0,-9.1,10.6,0.6,-10.0
4,10.6,-10.0,-10.0,-9.7,-9.7,-10.0,-9.7,10.3,10.3,10.3,...,0.3,-9.7,10.6,0.0,0.3,0.6,10.0,-10.0,10.0,0.0
5,10.0,10.9,0.3,-10.0,10.0,0.3,10.0,10.0,10.0,-9.4,...,10.6,10.0,-10.0,10.3,10.0,-9.7,-9.4,10.0,10.9,10.0
6,10.6,10.0,-10.0,0.3,-9.7,0.3,10.3,0.9,-9.4,0.0,...,10.0,-9.1,10.0,10.0,-9.1,11.2,10.3,0.6,-10.0,0.0
7,10.0,-9.7,0.3,10.3,10.0,10.3,0.6,0.9,-9.4,10.3,...,10.6,0.0,10.6,-9.1,10.3,10.3,10.3,0.6,10.0,0.0
8,10.3,0.6,10.0,10.6,-10.0,0.0,0.6,10.0,0.0,10.3,...,0.3,10.0,10.3,-10.0,10.0,-10.0,0.6,10.0,10.3,0.0
9,10.0,10.9,0.6,10.6,10.0,10.3,-10.0,10.0,0.3,10.9,...,10.6,10.6,0.6,10.3,10.6,10.6,-10.0,10.3,10.6,0.0


In [29]:
scores_df_clean = scores_df.copy().loc[:, ~(scores_df == 0).all()]

scores_df_clean

,0,1,2,3,4,5,6,7,8,9,...,343,344,345,346,347,348,349,350,351,352
0,10.0,10.6,-9.7,10.6,10.0,0.0,10.3,-9.7,0.0,0.6,...,10.0,10.3,10.6,0.0,-10.0,10.3,-10.0,10.3,10.3,0.0
1,10.6,10.3,0.3,-9.7,10.0,10.6,10.6,10.0,10.9,0.0,...,10.0,-9.4,0.0,10.0,10.3,10.0,10.6,-9.7,-9.4,0.0
2,0.0,0.3,10.3,0.0,0.6,0.3,0.6,10.0,10.3,0.3,...,0.0,10.3,-9.7,10.3,-9.7,-9.7,10.0,10.0,10.0,0.0
3,-10.0,-10.0,10.0,10.6,10.0,-10.0,-9.7,10.0,10.6,0.0,...,0.3,11.2,10.0,0.3,0.3,0.0,-9.1,10.6,0.6,-10.0
4,10.6,-10.0,-10.0,-9.7,-9.7,-10.0,-9.7,10.3,10.3,10.3,...,0.3,-9.7,10.6,0.0,0.3,0.6,10.0,-10.0,10.0,0.0
5,10.0,10.9,0.3,-10.0,10.0,0.3,10.0,10.0,10.0,-9.4,...,10.6,10.0,-10.0,10.3,10.0,-9.7,-9.4,10.0,10.9,10.0
6,10.6,10.0,-10.0,0.3,-9.7,0.3,10.3,0.9,-9.4,0.0,...,10.0,-9.1,10.0,10.0,-9.1,11.2,10.3,0.6,-10.0,0.0
7,10.0,-9.7,0.3,10.3,10.0,10.3,0.6,0.9,-9.4,10.3,...,10.6,0.0,10.6,-9.1,10.3,10.3,10.3,0.6,10.0,0.0
8,10.3,0.6,10.0,10.6,-10.0,0.0,0.6,10.0,0.0,10.3,...,0.3,10.0,10.3,-10.0,10.0,-10.0,0.6,10.0,10.3,0.0
9,10.0,10.9,0.6,10.6,10.0,10.3,-10.0,10.0,0.3,10.9,...,10.6,10.6,0.6,10.3,10.6,10.6,-10.0,10.3,10.6,0.0


In [30]:
scores_df_clean.replace(0, np.nan, inplace = True)

In [31]:
# Compute cosine similarity between all pairs of users
user_similarities = cosine_similarity(scores_matrix)


In [32]:
missings = np.where(scores_matrix == 0)

u_ids = missings[0]
c_ids = missings[1]

c_ids

array([  5,   8,  20, ..., 324, 351, 352], dtype=int64)

In [33]:
z = np.zeros((n_users, max_length))

for i in range(len(u_ids)):
    z[u_ids[i], c_ids[i]] = predict_rating(scores_matrix, u_ids[i], c_ids[i])
        
z

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        5.61627375],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        4.70234606],
       [4.05648745, 0.        , 0.        , ..., 0.        , 0.        ,
        3.00028909],
       ...,
       [4.12893527, 0.        , 0.        , ..., 0.        , 0.        ,
        1.01249556],
       [0.        , 0.        , 0.        , ..., 3.57455307, 0.        ,
        4.03971813],
       [0.        , 0.        , 0.        , ..., 0.        , 4.61198631,
        6.4542439 ]])

In [34]:
only_predictions = pd.DataFrame(z)

only_predictions

,0,1,2,3,4,5,6,7,8,9,...,343,344,345,346,347,348,349,350,351,352
0,0.000000,0.000000,0.0,0.000000,0.000000,5.425251,0.000000,0.0,6.564015,0.000000,...,0.000000,0.000000,0.000000,4.295848,0.000000,0.000000,0.000000,0.000000,0.000000,5.616274
1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3.944441,...,0.000000,0.000000,6.832542,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.702346
2,4.056487,0.000000,0.0,4.701692,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,2.625208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000289
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.475670,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.064420,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.356133,0.000000,0.000000,0.000000,0.000000,0.000000,4.105078
5,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3.382617,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.855432
7,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,2.747197,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.322166
8,0.000000,0.000000,0.0,0.000000,0.000000,5.796752,0.000000,0.0,6.923959,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.144820
9,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.905713


In [35]:
np.where(only_predictions != 0)

(array([ 0,  0,  0, ..., 30, 30, 30], dtype=int64),
 array([  5,   8,  20, ..., 324, 351, 352], dtype=int64))

In [36]:
unwatched_list = []

for i in range(len(only_predictions)):
    unwatched_list.append(list(np.where(z[i] != 0)[0]))
                     
unwatched_list

[[5,
  8,
  20,
  22,
  23,
  25,
  46,
  71,
  86,
  87,
  101,
  117,
  131,
  146,
  151,
  152,
  174,
  178,
  193,
  194,
  196,
  206,
  223,
  231,
  241,
  254,
  255,
  257,
  287,
  290,
  296,
  307,
  314,
  346,
  352],
 [9,
  24,
  29,
  33,
  68,
  73,
  84,
  101,
  120,
  121,
  129,
  150,
  158,
  176,
  179,
  184,
  207,
  209,
  220,
  238,
  250,
  262,
  275,
  281,
  310,
  321,
  332,
  339,
  341,
  345,
  352],
 [0,
  3,
  16,
  43,
  64,
  65,
  85,
  100,
  139,
  153,
  156,
  215,
  257,
  272,
  299,
  307,
  311,
  343,
  352],
 [9,
  17,
  18,
  32,
  45,
  52,
  59,
  74,
  83,
  110,
  136,
  149,
  162,
  163,
  169,
  192,
  201,
  205,
  206,
  208,
  212,
  227,
  243,
  250,
  258,
  262,
  263,
  265,
  280,
  292,
  297,
  301,
  305,
  348],
 [33,
  39,
  50,
  60,
  88,
  104,
  105,
  113,
  121,
  139,
  167,
  168,
  178,
  182,
  185,
  186,
  189,
  201,
  205,
  211,
  225,
  245,
  249,
  251,
  254,
  312,
  317,
  321,
  346,
  35

In [37]:
score_list = []
c = 0
for i in unwatched_list:
    s = []
    for j in i:
        s.append(z[c, j])
    score_list.append(s)
    c += 1
    
score_list

[[5.42525145347719,
  6.564015462752394,
  6.944798700006885,
  3.3711025138008153,
  3.024784028006922,
  2.882579466089111,
  2.0905190306222967,
  3.0954939007590245,
  3.092473836545725,
  3.0817341172355177,
  6.709775131351724,
  3.4078880449996785,
  1.8110803215733664,
  3.2740361028919303,
  4.392200841907588,
  5.075243066791775,
  3.7393437138185206,
  2.206347276749636,
  5.396386461524887,
  4.536770138197061,
  1.318944026609024,
  2.7888766657632242,
  4.086943965804976,
  5.011073693963998,
  2.8331473357439156,
  0.6356281971936678,
  3.03765122062595,
  4.44389423648948,
  -0.014740684840014355,
  2.5080630613308386,
  3.9955486906957653,
  3.6655584912947874,
  5.393642090229178,
  4.295847872043698,
  5.616273750803174],
 [3.944441475055498,
  -1.1166632954961544,
  2.7853688715228806,
  4.5515792088701765,
  3.636898034935866,
  5.765463280659433,
  3.5998121167193258,
  5.789967685603614,
  5.388778780659059,
  4.402552933015761,
  2.4651431827278345,
  6.18356075

In [38]:
score_indices = []
for i in score_list:
    order = sorted(range(len(i)), reverse = True, key=lambda k: i[k])
    score_indices.append(order)
    
score_indices

[[2,
  10,
  1,
  34,
  0,
  18,
  32,
  15,
  23,
  19,
  27,
  14,
  33,
  22,
  30,
  16,
  31,
  11,
  3,
  13,
  7,
  8,
  9,
  26,
  4,
  5,
  24,
  21,
  29,
  17,
  6,
  12,
  20,
  25,
  28],
 [29,
  11,
  7,
  5,
  24,
  8,
  27,
  30,
  3,
  23,
  9,
  17,
  0,
  16,
  4,
  6,
  12,
  13,
  15,
  25,
  2,
  22,
  21,
  10,
  28,
  26,
  14,
  18,
  19,
  20,
  1],
 [14, 11, 8, 4, 12, 7, 1, 13, 16, 2, 10, 0, 3, 15, 6, 18, 17, 9, 5],
 [24,
  7,
  0,
  16,
  33,
  18,
  6,
  4,
  10,
  14,
  25,
  1,
  22,
  11,
  32,
  20,
  12,
  15,
  5,
  17,
  19,
  26,
  31,
  8,
  13,
  23,
  29,
  28,
  27,
  3,
  30,
  9,
  2,
  21],
 [6,
  2,
  9,
  13,
  20,
  8,
  4,
  19,
  29,
  7,
  27,
  28,
  0,
  17,
  14,
  23,
  15,
  10,
  26,
  11,
  16,
  1,
  3,
  12,
  25,
  21,
  18,
  5,
  24,
  22],
 [1,
  15,
  24,
  21,
  6,
  0,
  7,
  11,
  19,
  3,
  8,
  10,
  4,
  17,
  25,
  9,
  12,
  16,
  5,
  20,
  13,
  22,
  14,
  2,
  23,
  18],
 [22,
  14,
  11,
  12,
  24,
  7,
  17,

In [39]:
rec_list = []
for i in range(len(score_indices)):
    l = unwatched_list[i]
    order = score_indices[i]
    rec_list.append([l[j] for j in order])
    
rec_list

[[20,
  101,
  8,
  352,
  5,
  193,
  314,
  152,
  231,
  194,
  257,
  151,
  346,
  223,
  296,
  174,
  307,
  117,
  22,
  146,
  71,
  86,
  87,
  255,
  23,
  25,
  241,
  206,
  290,
  178,
  46,
  131,
  196,
  254,
  287],
 [345,
  150,
  101,
  73,
  310,
  120,
  339,
  352,
  33,
  281,
  121,
  209,
  9,
  207,
  68,
  84,
  158,
  176,
  184,
  321,
  29,
  275,
  262,
  129,
  341,
  332,
  179,
  220,
  238,
  250,
  24],
 [299,
  215,
  139,
  64,
  257,
  100,
  3,
  272,
  311,
  16,
  156,
  0,
  43,
  307,
  85,
  352,
  343,
  153,
  65],
 [258,
  74,
  9,
  201,
  348,
  206,
  59,
  45,
  136,
  169,
  262,
  17,
  243,
  149,
  305,
  212,
  162,
  192,
  52,
  205,
  208,
  263,
  301,
  83,
  163,
  250,
  292,
  280,
  265,
  32,
  297,
  110,
  18,
  227],
 [105,
  50,
  139,
  182,
  225,
  121,
  88,
  211,
  352,
  113,
  321,
  346,
  33,
  201,
  185,
  251,
  186,
  167,
  317,
  168,
  189,
  39,
  60,
  178,
  312,
  245,
  205,
  104,
  254,
  24

In [40]:
# output dataframe with user ids a

recommendations_movies = pd.DataFrame(rec_list)

recommendations_movies.to_csv('movie_recommendations.csv', index = False)
recommendations_movies

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,20,101,8,352,5,193,314,152.0,231.0,194.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,345,150,101,73,310,120,339,352.0,33.0,281.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,299,215,139,64,257,100,3,272.0,311.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,258,74,9,201,348,206,59,45.0,136.0,169.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,50,139,182,225,121,88,211.0,352.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20,177,309,296,94,16,112,156.0,262.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,333,296,232,269,352,177,311,225.0,132.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,67,51,311,161,49,108,182,180.0,318.0,191.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,237,5,352,217,16,134,79.0,19.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,101,293,352,289,311,191,29,121.0,77.0,201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
pd.read_csv('movie_recommendations.csv')

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,20,101,8,352,5,193,314,152.0,231.0,194.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,345,150,101,73,310,120,339,352.0,33.0,281.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,299,215,139,64,257,100,3,272.0,311.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,258,74,9,201,348,206,59,45.0,136.0,169.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,50,139,182,225,121,88,211.0,352.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20,177,309,296,94,16,112,156.0,262.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,333,296,232,269,352,177,311,225.0,132.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,67,51,311,161,49,108,182,180.0,318.0,191.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,237,5,352,217,16,134,79.0,19.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,101,293,352,289,311,191,29,121.0,77.0,201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
